<a href="https://colab.research.google.com/github/toche7/PromptEngforDeveloper/blob/main/PromptLab8_ChatStreamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab Chatbot with Streamlit
เอกสารนี้พัฒนามาจากเอกสารของ Jeff Heaton, McKelvey School of Engineering, Washington University in St. Louis



ใน Lab นี้ เราจะได้สร้างแอปแชท LLM โดยใช้ StreamLit อย่างเป็นขั้นเป็นตอน เพื่อให้ทุกคนเข้าถึงและทำตามได้ง่าย โดยจะรันแอปนี้ผ่าน Google Colab เพื่อให้ใช้งานได้สะดวกขึ้น อีกทั้งเราจะพาไปรู้จักกับ `llm_util.py` ซึ่งเป็นสคริปต์ยูทิลิตี้ที่ช่วยให้ทำงานกับโมเดลภาษาขนาดใหญ่ (LLMs) ที่รองรับ LangChain ได้ง่ายยิ่งขึ้น ในตัวอย่างนี้ เราจะใช้ LLM ของ Meta เป็นตัวขับเคลื่อนสำหรับแอปแชทของเรา เมื่อจบโมดูลนี้ คุณจะได้แอปแชทที่ใช้งานได้จริง พร้อมความเข้าใจพื้นฐานในการผสาน LLM เข้าไปในโปรเจกต์ต่าง ๆ โดยใช้ StreamLit

โค้ดต่อไปนี้เริ่มต้นด้วยการนำเข้าไลบรารี `os` เพื่อจัดการกับระบบไฟล์และตัวแปรสภาพแวดล้อม จากนั้นตรวจสอบว่าโค้ดรันใน Google Colab หรือไม่ โดยพยายามนำเข้าโมดูลจาก `google.colab` ถ้านำเข้าได้ จะตั้งค่าตัวแปร `COLAB` เป็น `True` และพิมพ์ข้อความแจ้งว่ากำลังใช้ Colab แต่ถ้าไม่สามารถนำเข้าได้ จะตั้งค่าเป็น `False` แทน

ถ้าอยู่ใน Colab จะตั้งค่าตัวแปรสภาพแวดล้อม `GROQ_API_KEY` จากข้อมูลที่เก็บไว้ใน Colab แล้วติดตั้งไลบรารีที่จำเป็นสำหรับการทำงาน เช่น `langchain`, `langchain_openai`, `openai`, `streamlit`, และ `langchain-groq` ผ่านคำสั่ง pip โดยรวมโค้ดนี้ทำให้สามารถใช้งานได้อย่างเหมาะสมทั้งใน Google Colab และสภาพแวดล้อมอื่น ๆ

In [ ]:
import os

try:
    from google.colab import drive, userdata
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# Groq Secrets
if COLAB:
    os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

# Install needed libraries in CoLab
if COLAB:
    !pip install langchain langchain_openai openai streamlit langchain-groq

# Creating a Chat Application

เราจะสร้างไฟล์ดังนี้:

* **app.py** - แอปพลิเคชันหลักของ StreamLit สำหรับแชท


## Chat Application
โค้ดนี้สร้างแอปพลิเคชันแชทบอทโดยใช้ Streamlit และโมเดลภาษาจาก `ChatGroq` ในไลบรารี LangChain โดยการตั้งค่าเริ่มต้นและรูปแบบการทำงานจะอยู่ในไฟล์ชื่อ `app.py` โดยโค้ดมีรายละเอียดดังนี้:

1. **การนำเข้าโมดูล**:
   - นำเข้า `ChatGroq` จาก `langchain_groq` สำหรับการทำงานกับโมเดลภาษา
   - นำเข้า Streamlit (`st`) เพื่อสร้างอินเทอร์เฟซผู้ใช้
   - นำเข้า `ConversationSummaryMemory`, `PromptTemplate`, และ `ConversationChain` จาก LangChain ซึ่งใช้สำหรับจัดการหน่วยความจำ, การสร้างเทมเพลตคำถาม, และจัดการการสนทนา

2. **การตั้งค่าโมเดลภาษา (LLM)**:
   - โค้ดกำหนดตัวแปร `llm` เป็นอินสแตนซ์ของ `ChatGroq` และระบุโมเดลภาษาที่จะใช้คือ `"gemma2-9b-it"`

3. **ฟังก์ชัน `create_chatbot`**:
   - ฟังก์ชันนี้ใช้สร้างแชทบอทที่มีหน่วยความจำ (`ConversationSummaryMemory`) ซึ่งช่วยให้แชทบอทสามารถติดตามบริบทของการสนทนา
   - มีการกำหนด `template` สำหรับแชทบอท ซึ่งเริ่มต้นด้วยข้อความของระบบว่า “System: You are a helpful assistant. Answer questions clearly and concisely.” เพื่อให้แชทบอททำงานตามบทบาทที่กำหนดไว้
   - ตัวเทมเพลตประกอบด้วย `{history}` สำหรับประวัติการสนทนาและ `{input}` สำหรับข้อความที่ผู้ใช้ป้อน จากนั้นคืนค่าเป็น `ConversationChain` ที่ประกอบไปด้วยโมเดลภาษา, เทมเพลต, และหน่วยความจำ เพื่อให้แชทบอทสามารถใช้ข้อมูลบริบทจากการสนทนาเดิมได้

4. **การตั้งค่า Streamlit**:
   - แสดงหัวข้อแอปเป็น "Chat" โดยใช้ `st.title("Chat")`
   - เช็คว่าใน `st.session_state` มีอินสแตนซ์ของแชทบอทหรือยัง ถ้ายังไม่มี จะสร้างแชทบอทใหม่โดยเรียก `create_chatbot`
   - เช็คสถานะของ `st.session_state.messages` เพื่อเก็บประวัติการสนทนา ถ้ายังไม่มีจะสร้างเป็นลิสต์ว่าง ๆ

5. **การแสดงประวัติการสนทนา**:
   - ลูปผ่าน `st.session_state.messages` และแสดงข้อความจากผู้ใช้และแชทบอททีละข้อความบนหน้าจอแชท

6. **การรับข้อความจากผู้ใช้และตอบกลับ**:
   - เมื่อผู้ใช้กรอกข้อความในช่องอินพุต ข้อความนั้นจะถูกเพิ่มเข้าในประวัติการสนทนาและแสดงในอินเทอร์เฟซ
   - แชทบอทประมวลผลข้อความที่ได้รับโดยใช้ฟังก์ชัน `predict` ของ `ConversationChain` และสร้างข้อความตอบกลับ จากนั้นแสดงในส่วนของผู้ช่วยและบันทึกลงในประวัติการสนทนา

โดยรวม โค้ดนี้สร้างแอปพลิเคชันแชทบอทที่มีระบบหน่วยความจำและบริบทเพื่อให้การสนทนาสอดคล้องและต่อเนื่อง


In [ ]:
%%writefile app.py
from langchain_groq import ChatGroq
import streamlit as st
from langchain.memory import ConversationSummaryMemory
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import ConversationChain
import sys

llm = ChatGroq(
    model="gemma2-9b-it",
)

def create_chatbot(llm):
    memory = ConversationSummaryMemory(llm=llm)

    # Define the system prompt directly in the template
    template = """System: You are a helpful assistant. Answer questions clearly and concisely.

{history}
User: {input}

Assistant:
"""
    PROMPT = PromptTemplate(input_variables=["history", "input"], template=template)
    return ConversationChain(llm=llm, prompt=PROMPT, memory=memory, verbose=False)


st.title("Chat")

# Initialize the chat and messages
if "chat" not in st.session_state:
    st.session_state.chat = create_chatbot(llm)

if "messages" not in st.session_state:
    st.session_state.messages = []

# Display the conversation history
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(
            message["content"],
        )

# Process user input and chatbot response
if prompt := st.chat_input("What is up?"):
    st.session_state.messages.append({"role": "user", "content": prompt})
    with st.chat_message("user"):
        st.markdown(
            prompt,
        )

    with st.chat_message("assistant"):
        response = st.session_state.chat.predict(input=prompt)
        st.markdown(
            response,
        )
    st.session_state.messages.append({"role": "assistant", "content": response})


เราต้องไปเอา password จากการใช้ lib localtunnel เพื่อการเข้าถึงการใช้งาน server จากภายนอกเพื่อทำให้ run Streamlit ใน Colab ได้  

ตัวเลข IP ที่ได้นี้จะเป็น password ที่เราจะนำไปใช้ใน step ต่อไป

In [ ]:
!curl https://loca.lt/mytunnelpassword

We launch the StreamLit server and obtain its URL. You will need the above password when you access the URL it gives you.

เราจะทำการขึ้น StreamLit Server และได้ URL ของมัน กดไปที่ link ที่ได้จากการ run คำสั่งด้านล่างนี้

In [ ]:
!streamlit run app.py &>/content/logs.txt &
!npx --yes localtunnel --port 8501

คำสั่งนี้เป็นการใช้ใน Google Colab เพื่อรันแอปพลิเคชัน Streamlit และเชื่อมต่อกับ LocalTunnel ดังนี้:

1. **รันแอป Streamlit**:
   - `!streamlit run app.py &>/content/logs.txt &`
   - คำสั่งนี้จะรันไฟล์ Python ที่ชื่อ `app.py` โดยใช้ Streamlit
   - `&>` ใช้เพื่อส่งออกผลลัพธ์ทั้ง stdout และ stderr ไปยังไฟล์ `logs.txt` ในโฟลเดอร์ `/content`
   - `&` ที่ท้ายคำสั่งหมายความว่าการรันจะเกิดขึ้นในพื้นหลัง (background) ทำให้คุณสามารถรันคำสั่งถัดไปได้โดยไม่ต้องรอให้ Streamlit เสร็จสิ้น

2. **เชื่อมต่อกับ LocalTunnel**:
   - `!npx --yes localtunnel --port 8501`
   - คำสั่งนี้จะใช้ `npx` เพื่อเรียกใช้งาน LocalTunnel ซึ่งจะทำการสร้าง URL สาธารณะเพื่อเข้าถึงแอป Streamlit ที่รันอยู่ที่พอร์ต 8501
   - `--yes` จะยืนยันการติดตั้งแพ็คเกจที่จำเป็น

